In [1]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Initialize()
ee.Authenticate

<function ee.Authenticate(authorization_code: Optional[str] = None, quiet: Optional[bool] = None, code_verifier: Optional[str] = None, auth_mode: Optional[str] = None, scopes: Optional[Sequence[str]] = None, force: bool = False) -> Optional[bool]>

In [2]:
shapefile_path = "C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp"

# Check if the path is a .zip file
if shapefile_path.endswith('.zip'):
    # Try to read shapefile from a zip archive
    try:
        # Check if the .zip file exists and open it
        with zipfile.ZipFile(shapefile_path, 'r') as zip_ref:
            zip_ref.printdir()  # Optional: Print contents of the zip to debug
            # Try to find the .shp file inside the zip
            shapefile_found = False
            for file in zip_ref.namelist():
                if file.endswith('.shp'):
                    shapefile_found = True
                    shapefile_within_zip = file
                    break

            if shapefile_found:
                # Read shapefile directly from the zip file
                oi = gpd.read_file(f'zip://{shapefile_path}/{shapefile_within_zip}')
                print(f"Successfully loaded shapefile from {shapefile_path}.")
            else:
                print("No .shp file found inside the zip archive.")
                #
    except Exception as e:
        print(f"Error reading shapefile from zip archive: {e}")
        #
else:
    # If not a .zip, assume it is a regular shapefile
    try:
        # Read the shapefile normally
        aoi = gpd.read_file(shapefile_path)
        print(f"Successfully loaded shapefile from {shapefile_path}.")
    except Exception as e:
        print(f"Error reading shapefile: {e}")


# After loading, check if the GeoDataFrame is not empty
if not aoi.empty:
    # If the GeoDataFrame contains multiple geometries, dissolve them into one
    if len(aoi) > 1:
        aoi = aoi.dissolve()

    # Extract the first geometry from the dissolved GeoDataFrame
    geometry = aoi.geometry.iloc[0]

    # Check if the geometry is a Polygon or MultiPolygon
    if geometry.geom_type in ['Polygon', 'MultiPolygon']:
        # Convert the geometry to GeoJSON format
        geojson = geometry.__geo_interface__

        # Remove the third dimension from the coordinates if it exists
        if geojson['type'] == 'Polygon':
            geojson['coordinates'] = [list(map(lambda coord: coord[:2], ring)) for ring in geojson['coordinates']]
        elif geojson['type'] == 'MultiPolygon':
            geojson['coordinates'] = [[list(map(lambda coord: coord[:2], ring)) for ring in polygon] for polygon in geojson['coordinates']]

        # Create an Earth Engine geometry object from the GeoJSON coordinates
        ee_geometry = ee.Geometry(geojson)

        # Convert the Earth Engine geometry to a Feature
        feature = ee.Feature(ee_geometry)

        # Create a FeatureCollection with the feature
        aoi = ee.FeatureCollection([feature])

        print("AOI defined successfully.")

        # check_next_button()
    else:
        
        print("The geometry is not a valid type (Polygon or MultiPolygon).")
else:
    print("The shapefile does not contain any geometries.")        #

Successfully loaded shapefile from C:/Users/pc/My Drive/2025/Uni/TCC/Shapes/contorno_area_total/contorno_area_total.shp.
AOI defined successfully.


In [3]:
# Define the start and end dates for filtering the image collection
inicio = '2022-01-01'
final = '2023-12-31'
nuvem ='40'

# Define the start and end dates for filtering the image collection
startDate = '2022-01-01'
endDate = '2023-12-31'

# Load the Sentinel-2 image collection and filter by date, location, and cloud coverage
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(startDate, endDate) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))

# Get the number of images in the collection
count = sentinel2.size().getInfo()
print(f"Number of images in collection: {count}")

Number of images in collection: 297


In [5]:
#Coverage Ratio Function
# -------------------------------
aoi_geometry = aoi.first().geometry()
aoi_area = aoi_geometry.area()

coverage_threshold = 1

def calculate_coverage_ratio(image):
    """
    Calculates the ratio of the AOI area covered by the image.
    
    Args:
        image (ee.Image): The Sentinel-2 image.
    
    Returns:
        ee.Image: The original image with an added 'coverage_ratio' property.
    """
    # Compute the intersection geometry between AOI and image footprint
    intersection = aoi_geometry.intersection(image.geometry(), ee.ErrorMargin(1))
    
    # Calculate the area of the intersection
    intersection_area = intersection.area()
    
    # Calculate the coverage ratio (intersection area / AOI area)
    coverage_ratio = intersection_area.divide(aoi_area)
    
    # Set the coverage ratio as a property of the image
    return image.set('coverage_ratio', coverage_ratio)

# -------------------------------
# Step 6: Apply Coverage Ratio Calculation
# -------------------------------

# Map the coverage ratio function over the Sentinel-2 collection
sentinel2_with_ratio = sentinel2.map(calculate_coverage_ratio)

# -------------------------------
# Step 7: Filter Based on Coverage Ratio
# -------------------------------

# Define a filter to keep images with coverage_ratio >= coverage_threshold
coverage_filter = ee.Filter.gte('coverage_ratio', coverage_threshold)

# Apply the filter to get the final collection
fully_covering_images = sentinel2_with_ratio.filter(coverage_filter)

# Get the number of images before filtering
initial_count = sentinel2.size().getInfo()

# Get the number of images after coverage filtering
filtered_count = fully_covering_images.size().getInfo()

print(f"Number of images before coverage filtering: {initial_count}")
print(f"Number of images with >= {coverage_threshold*100}% AOI coverage: {filtered_count}")


Number of images before coverage filtering: 297
Number of images with >= 100% AOI coverage: 250


In [5]:
sentine2 = filtered_count